In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import awoc
import itertools

In [2]:
col_air = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 
           'Altitude', 'Timezone', 'DST', 'TZ', 'Type', 'Source']
airports = pd.read_csv('airports.csv', delimiter=',', names = col_air)
airports.head(5)


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [3]:
col_rout = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 
            'Codeshare', 'Stops', 'Equipment']
routes  = pd.read_csv('routes.csv', names = col_rout)
routes.head(3)

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


In [4]:
def country_to_continent(name):
    try:
        continent = awoc.AWOC().get_country_continent_name(name)
    except NameError:
        continent = np.nan
    return continent

continents = airports.Country.map(lambda x: country_to_continent(x))
airports['Continent'] = continents

dict_territory = {"Cote d'Ivoire": 'Africa', 'Congo (Brazzaville)': 'Africa', 'Congo (Kinshasa)': 'Africa',
        'Midway Islands': 'North America', 'French Guiana': 'South America', 'Martinique': 'North America', 
        'Guadeloupe': 'North America','Virgin Islands': 'North America', 'Burma': 'Asia', 'Norfolk Island': 'Oceania', 
        'Johnston Atoll': 'North America','Cocos (Keeling) Islands': 'Asia', 'Wake Island': 'North America', 'West Bank': 'Asia', 'Svalbard': 'Europe' 
}
airports.Continent = airports.Continent.fillna(airports['Country'].map(dict_territory))

## 3 крупнейших аэропорта для каждой страны. Сортировка по 2 наибольшему

In [5]:
list_of_countries = airports[airports.Continent == 'Europe'].Country.unique()
number_of_countries = len(list_of_countries)
number_of_countries # Все страны на континенте Европа

46

In [6]:
df1 = airports[airports.IATA.notna()]

In [7]:
routes_source  = routes[['Airline ID', 'Source airport']].groupby('Source airport').count()
routes_source.index.name = 'IATA'
routes_source.columns = ['Source']
routes_destin = routes[['Airline ID', 'Destination airport']].groupby('Destination airport').count()
routes_destin.index.name = 'IATA'
routes_destin.columns = ['Destination']
routes_amount = routes_source.join(routes_destin)
routes_amount['Sum'] = routes_amount.sum(axis = 1)

In [8]:
df1 = pd.merge(df1, routes_amount, left_on = 'IATA', right_index = True)

In [10]:
first_3 = df1[['Name', 'Country', 'Sum']].groupby(['Country'], ).apply(lambda x: x.nlargest(3, 'Sum').sort_values('Sum', ascending=False))
index = pd.MultiIndex.from_frame(first_3[['Country', 'Name']])
first_3 = pd.DataFrame(first_3['Sum'])
first_3.index = index
first_3.head() # 2-й пункт - сортировка по названию страны и внутри страны от наииеньшего к наибольшему 

Sum
Country     Name                                             
Afghanistan Kabul International Airport                  54.0
            Herat Airport                                12.0
            Kandahar Airport                             10.0
Albania     Tirana International Airport Mother Teresa   72.0
Algeria     Houari Boumediene Airport                   176.0

In [11]:
def key_sort(key, df):
    if df.loc[key].count()[0] > 1:
        return int(df.loc[key].iloc[1].values)
    else:
        return int(df.loc[key].values[0])
indexies = first_3.index
sort_indexies = sorted(indexies, key = lambda x: key_sort(x[0], first_3), reverse=True)
sort_indexies
sort_by_2biggest = first_3.loc[sort_indexies]
sort_by_2biggest.head(10) #3-й пункт - Сортировка по 2-му наибольшему аэропорту, внутри страны - от наибольшего к наименьшему

Sum
Country       Name                                                    
United States Hartsfield Jackson Atlanta International Airport  1826.0
              Chicago O'Hare International Airport              1108.0
              Los Angeles International Airport                  990.0
China         Beijing Capital International Airport             1069.0
              Shanghai Pudong International Airport              825.0
              Guangzhou Baiyun International Airport             674.0
Singapore     Singapore Changi Airport                           820.0
Germany       Frankfurt am Main International Airport            990.0
              Munich International Airport                       728.0
              Düsseldorf International Airport                   570.0

In [12]:
translate_countries = np.unique(list(map(lambda x: x[0], sort_indexies)))
translate_countries

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Eston

In [13]:
text = open('translate.txt', encoding="utf8")
translate = text.readlines()
text.close()
for i in range(len(translate)):
    translate[i] = translate[i].strip()[1:-1].replace('"', "'").rstrip("'").split("', '")
translate = list(itertools.chain.from_iterable(translate))

In [14]:
dict_list = {}
for i in list(zip(translate_countries, translate)):
    dict_list[i[0]] = i[1]
dict_list

{'Afghanistan': 'Афганистан',
 'Albania': 'Албания',
 'Algeria': 'Алжир',
 'American Samoa': 'Американское Самоа',
 'Angola': 'Ангола',
 'Anguilla': 'Ангилья',
 'Antigua and Barbuda': 'Антигуа и Барбуда',
 'Argentina': 'Аргентина',
 'Armenia': 'Армения',
 'Aruba': 'Аруба',
 'Australia': 'Австралия',
 'Austria': 'Австрия',
 'Azerbaijan': 'Азербайджан',
 'Bahamas': 'Багамы',
 'Bahrain': 'Бахрейн',
 'Bangladesh': 'Бангладеш',
 'Barbados': 'Барбадос',
 'Belarus': 'Беларусь',
 'Belgium': 'Бельгия',
 'Belize': 'Белиз',
 'Benin': 'Бенина',
 'Bermuda': 'Бермуды',
 'Bhutan': 'Бутан',
 'Bolivia': 'Боливии',
 'Bosnia and Herzegovina': 'Босния и Герцеговина',
 'Botswana': 'Ботсвана',
 'Brazil': 'Бразилия',
 'British Virgin Islands': 'Британские Виргинские Острова',
 'Brunei': 'Брунейский',
 'Bulgaria': 'Болгария',
 'Burkina Faso': 'Буркина-Фасо',
 'Burma': 'Бирмы',
 'Burundi': 'Бурундийский',
 'Cambodia': 'Камбоджа',
 'Cameroon': 'Камерун',
 'Canada': 'Канада',
 'Cape Verde': 'Зеленый Мыс',
 'Caym

In [15]:
rus_version = first_3.copy(deep=True)
rus_index = pd.MultiIndex.from_tuples(list(map(lambda x: (dict_list[x[0]], x[1]), rus_version.index)), names=["Cointry", "Name"])
rus_version.index = rus_index
rus_index = sorted(rus_index , key = lambda x: x[0])
rus_version = rus_version.loc[rus_index]
rus_version.head(10) # Таблица из пункта 2 на русском

Sum
Cointry     Name                                               
Австралия   Sydney Kingsford Smith International Airport  410.0
            Brisbane International Airport                296.0
            Melbourne International Airport               269.0
Австрия     Vienna International Airport                  616.0
            Salzburg Airport                               69.0
            Graz Airport                                   38.0
Азербайджан Heydar Aliyev International Airport           128.0
            Ganja Airport                                   9.0
            Lankaran International Airport                  4.0
Албания     Tirana International Airport Mother Teresa     72.0

## График авиаперелетов Для Греции

In [16]:
airports_clean = airports.apply(lambda col: col.replace('\\N', np.nan)).dropna(subset = ['IATA'])

In [17]:
Greece_df = airports_clean[(airports_clean.Country == 'Greece')&(airports_clean.IATA.notna())] # датафрейм с аэропортами Греции имеющими код IATA

In [18]:
Greece_IATA = np.array(Greece_df.IATA)

In [19]:
Greece_routes = routes[(routes['Source airport'].map(lambda x: x in Greece_IATA))
                       & (routes['Destination airport'].map(lambda x: x not in Greece_IATA))][['Source airport', 'Destination airport', 'Airline']]

In [20]:
Greece_routes = Greece_routes.groupby(['Source airport', 'Destination airport'], as_index = False).count()
Greece_routes = Greece_routes.rename(columns={Greece_routes.columns[2]: "Number of flights" })
Greece_routes

,Source airport,Destination airport,Number of flights
0,ATH,ADB,1
1,ATH,AMM,1
2,ATH,AMS,2
3,ATH,ARN,2
4,ATH,AUH,2
...,...,...,...
351,ZTH,LBA,1
352,ZTH,LGW,2
353,ZTH,MAN,2
354,ZTH,MUC,1


In [21]:
place_of_air = airports[['Latitude', 'Longitude']]
place_of_air.index = airports['IATA']
Greece_routes = Greece_routes.merge(place_of_air, left_on = 'Destination airport', right_index=True)
Greece_routes = Greece_routes.merge(place_of_air, left_on = 'Source airport', right_index=True)
Greece_routes = Greece_routes.rename(columns={
                     Greece_routes.columns[3]: "Latitude destination",
                     Greece_routes.columns[4]:'Longitude destination',
                     Greece_routes.columns[5]: "Latitude source",
                     Greece_routes.columns[6]:'Longitude source',})
Greece_routes = Greece_routes.reset_index()
Greece_routes.head(10) # Таблица аэропорт вылета - аэропорт прибытия - широта и долгота аэропортов


,index,Source airport,Destination airport,Number of flights,Latitude destination,Longitude destination,Latitude source,Longitude source
0,0,ATH,ADB,1,38.292400,27.157000,37.936401,23.9445
1,1,ATH,AMM,1,31.722601,35.993198,37.936401,23.9445
2,2,ATH,AMS,2,52.308601,4.763890,37.936401,23.9445
3,3,ATH,ARN,2,59.651901,17.918600,37.936401,23.9445
4,4,ATH,AUH,2,24.433001,54.651100,37.936401,23.9445
5,5,ATH,BCN,4,41.297100,2.078460,37.936401,23.9445
6,6,ATH,BEG,3,44.818401,20.309099,37.936401,23.9445
7,7,ATH,BEY,1,33.820900,35.488400,37.936401,23.9445
8,8,ATH,BGY,1,45.673901,9.704170,37.936401,23.9445
9,9,ATH,BRU,2,50.901402,4.484440,37.936401,23.9445


## Строим карту международных авиаперелетов для аэропортов Греции

In [22]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [35]:
airports = airports.replace('\\N', np.nan)
airports[(airports.Country == 'Greece') & (airports.IATA.isna())] = airports[(airports.Country == 'Greece') & (airports.IATA.isna())].replace(np.nan, 'No')

In [36]:
routes

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [37]:
all_Greece = airports[airports.Country == 'Greece'].IATA.unique()
int_air = routes.merge(airports[['IATA','Country']], left_on = 'Source airport', right_on = 'IATA')
int_air = int_air.merge(airports[['IATA','Country']], left_on = 'Destination airport', right_on = 'IATA')
int_air = int_air[int_air['Country_x'] != int_air['Country_y']]
int_air = int_air['Source airport'].unique()
airports = airports.dropna(subset = ['IATA'])
int_air

array(['DWC', 'FEG', 'SKD', ..., 'KSA', 'PNI', 'KWA'], dtype=object)

In [38]:
def key_to_col (air, set_1, set_2 = None):
    if air in set_1:
        return('red')
    elif air in set_2:
        return('green')
    else:
        return('blue')
airports_color = airports.IATA.map(lambda air: key_to_col(air, all_Greece, int_air)) 

In [39]:
#Здесь на карте обозначены:
#Аэропорты Греции - красным цветом
#Международные аэропорты - зеленым цветом
#Аэропорты с кодом IATA - синим цветом
fig = go.Figure(go.Scattermapbox(lat = airports['Latitude'], lon = airports['Longitude'],
               marker=dict(color=airports_color)))
fig.update_layout(mapbox_style="open-street-map")

fig.show()

In [41]:
#теперь раскрасим аэропорты Греции в зависимости от параметров
# Аэропорт осуществляющий международные рейсы - красный
# Аэропорты имеющие код IATA - зеленый
# Остальные аэропорты - синий
all_Greece_air = airports[airports.Country == 'Greece']
all_international = np.array(Greece_routes['Source airport'].unique())
airports_col = all_Greece_air.IATA.map(lambda air: key_to_col(air, all_international, np.array(all_Greece_air.IATA[all_Greece_air.IATA != 'No'])))
fig = go.Figure(go.Scattermapbox(lat = all_Greece_air['Latitude'], lon = all_Greece_air['Longitude'],
               marker=dict(color=airports_col)))
map_center = map_center = go.layout.mapbox.Center(lat = all_Greece_air.Latitude.iloc[0], lon = all_Greece_air.Longitude.iloc[0])
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=4))
fig.show()

In [42]:
all_lat = Greece_routes[['Latitude destination', 'Latitude source']].apply(lambda x: np.array(x), axis = 'columns')
all_long = Greece_routes[['Longitude destination', 'Longitude source']].apply(lambda x: np.array(x), axis = 'columns')
lines = Greece_routes[['Source airport', 'Number of flights']].join(pd.DataFrame(zip(all_lat, all_long), 
                                                                         columns = ['all_lat', 'all_long']))

In [43]:
#Сеть авиарейсов для каждого аэропорта Греции
#Легенда цветов (количество рейсов в один аэропорт - цвет):
# 1 рейс- фиолетовый
# 2 рейса - розовый
# 3 рейса - зеленый
# 4 рейса - золотой
# 5 рейсов - оранжевый
# 6 рейсов - красный
dict_nflights = {1: 'slateblue', 2: 'hotpink', 3: 'forestgreen', 4: 'gold', 5: 'orange',6: 'red'}
for air in all_international:
    df = lines[lines['Source airport'] == air]
    fig = go.Figure(go.Scattermapbox(lat = all_Greece_air['Latitude'], lon = all_Greece_air['Longitude'],
                                             marker=dict(
                                             color=airports_col)))
    for line in range(df.shape[0]):
        lat = df.iloc[line, 2] 
        long = df.iloc[line, 3] 
        color = dict_nflights[df.iloc[line, 1]]
        fig.add_trace(go.Scattermapbox(mode = "lines",
                               hoverinfo='skip',
                               lat=lat,
                               lon=long,
                               marker=dict(color=color)))
    fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=3))
    fig.show()